In [32]:
import pandas as pd
import numpy as np

In [33]:
%%time
# Importing the dataset
df = pd.read_csv('clean_dataset.csv')
df.rename(columns = {'text_clean':'text'}, inplace = True)

Wall time: 1.88 s


In [34]:
df.head()

,sentiment,text
0,0,aw buyer got david car third day
1,0,upset update facebook texting might cry result...
2,0,dived many time bal managed save rest go bound
3,0,whole body feel itch like fire
4,0,behaving al mad see


In [41]:
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

In [57]:
%%time
train, test = train_test_split(df, test_size=0.2, random_state=10)

Wall time: 1.7 s


In [74]:
%%time
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train.text)
vocab_size = len(tokenizer.word_index) + 1

Wall time: 25.8 s


In [78]:
%%time
#sequences_train = tokenizer.texts_to_sequences(train.text) 
#sequences_train = pad_sequences(sequences_train)

Wall time: 0 ns


In [83]:
%%time
sequences_train = tokenizer.texts_to_sequences(train.text) 
sequences_test = tokenizer.texts_to_sequences(test.text) 

X_train = pad_sequences(sequences_train)
X_test = pad_sequences(sequences_test)

y_train = train.sentiment.values
y_test = test.sentiment.values

Wall time: 35.4 s


In [99]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional

In [106]:
%%time
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=X_train.shape[1]))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 34, 128)           30048512  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_13 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 1026      
Total params: 30,444,290
Trainable params: 30,444,290
Non-trainable params: 0
_________________________________________________________________
Wall time: 922 ms


In [107]:
history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test), verbose=2)

Epoch 1/10


ValueError: in user code:

    C:\Users\maloh\anaconda3\lib\site-packages\keras\engine\training.py:830 train_function  *
        return step_function(self, iterator)
    C:\Users\maloh\anaconda3\lib\site-packages\keras\engine\training.py:813 run_step  *
        outputs = model.train_step(data)
    C:\Users\maloh\anaconda3\lib\site-packages\keras\engine\training.py:771 train_step  *
        loss = self.compiled_loss(
    C:\Users\maloh\anaconda3\lib\site-packages\keras\engine\compile_utils.py:201 __call__  *
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    C:\Users\maloh\anaconda3\lib\site-packages\keras\losses.py:142 __call__  *
        losses = call_fn(y_true, y_pred)
    C:\Users\maloh\anaconda3\lib\site-packages\keras\losses.py:246 call  *
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    C:\Users\maloh\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    C:\Users\maloh\anaconda3\lib\site-pac

In [ ]:
#History for accuracy
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train accuracy', 'Test accuracy'], loc='lower right')
plt.show()
# History for loss
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train loss', 'Test loss'], loc='upper right')
plt.suptitle('Accuracy and loss for second model')
plt.show()

In [ ]:
vec = CountVectorizer()
xeval=["I hate when I have to call and wake people up "]
print(xeval)
xeval_numeric = vec.fit_transform(xeval).toarray() 
prediction=model.predict(xeval_numeric)
print(prediction)